In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from offer_search.ranking.elasticsearch import ElasticsearchRanker

In [3]:
import pandas as pd

In [4]:
preset = []

In [5]:
def fill_preset(data, meta, col='Название'):
    for _, row in data.iterrows():
        for _, row_meta in meta.iterrows():
            body = {
                'Item': row[col].lower(),
                'Attributes': '' if row['Описание'] != row['Описание'] else row['Описание'],
                'Price': int(row['Цена']),
                'Offer': row_meta['NAME'],
                'Web': row_meta['WEB'],
                'Offer': row_meta['NAME'],
                'Cashback': 0 if row_meta['CASH_BACK_HEIGHT'] != row_meta['CASH_BACK_HEIGHT'] else row_meta['CASH_BACK_HEIGHT'],
                'Period': 0 if row_meta['TRANCHE_STMT_COUNT'] != row_meta['TRANCHE_STMT_COUNT'] else row_meta['TRANCHE_STMT_COUNT'],
                'Offer_type': row_meta['OFFER_TYPE'],
                'Advert_text': row_meta['ADVERT_TEXT']
            }
            preset.append(body)

In [6]:
data = pd.read_excel('./data/sport/double-sports/data_cleaned.xls')
data.head()

,Название,Описание,Цена
0,Велосипеды / Велосипеды / Туринги / AUTHOR / В...,Хромолевый грэвел (gravel) Author RONIN 2019 г...,84000
1,Велосипеды / Велосипеды / Горные / BERGAMONT /...,"Универсальный горный велосипед, который отличн...",43450
2,Велосипеды / Велосипеды / Горные / BERGAMONT /...,"Универсальный горный велосипед, который отличн...",43450
3,Велосипеды / Велосипеды / Горные / BERGAMONT /...,В 2018 году компания Bergamont решила объедини...,43450
4,Велосипеды / Велосипеды / Горные / BERGAMONT /...,В 2018 году компания Bergamont решила объедини...,43450


In [7]:
meta = pd.read_excel('./data/sport/double-sports/meta.xls')
meta.head()

,NAME,WEB,CASH_BACK_HEIGHT,TRANCHE_STMT_COUNT,OFFER_TYPE,ADVERT_TEXT
0,double sports,https://double-sports.ru/,0,6,SPECIAL_CREDIT,На покупку по кредитной карте не будут начисля...


In [8]:
fill_preset(data, meta)

In [9]:
data = pd.read_excel('./data/sport/gold-standart.com/data_cleaned.xls')
data.head()

,Наименование,Описание,Цена
0,Наборы спортивного питания~Комбо-набор Mad God...,MAD DRUGS GOD OF WHEY.Размер порции: 2 мерные ...,2200
1,Наборы спортивного питания~Комбо-набор 2 Ligan...,Спортивное питание FrogTechLigandrol (Лигандро...,2890
2,Наборы спортивного питания~Комбо-набор Ligandr...,Спортивное питание FrogTechIbutamoren (Ибутамо...,3990
3,Наборы спортивного питания~Комбо-набор 2 Ligan...,Спортивное питание FrogTechLigandrol (Лигандро...,4990
4,Наборы спортивного питания~Комбо-набор Ligandr...,Спортивное питание FrogTechIbutamoren (Ибутамо...,4990


In [10]:
meta = pd.read_excel('./data/sport/gold-standart.com/meta.xls')
meta.head()

,NAME,WEB,CASH_BACK_HEIGHT,TRANCHE_STMT_COUNT,OFFER_TYPE,ADVERT_TEXT
0,Gold-standart.com,https://gold-standart.com/,9,NaN,STANDARD,Gold-Standart.com это сеть магазинов спортивно...


In [11]:
fill_preset(data, meta, col='Наименование')

In [12]:
data = pd.read_excel('./data/sport/pro-bike/data_cleaned.xls')
data.head()

,Название,Описание,Цена
0,Велосипеды~Горный велосипед~Alpine Bike 1000S ...,NaN,11490
1,Велосипеды~Детский велосипед~Alpine Bike F1 (2...,NaN,20490
2,Велосипеды~Горный велосипед~Stels Navigator 66...,NaN,17400
3,Велосипеды~Дорожный велосипед~Schwinn Alu 7 Wo...,NaN,19500
4,Велосипеды~Детский велосипед~Schwinn Baywood 2...,NaN,10700


In [13]:
meta = pd.read_excel('./data/sport/pro-bike/meta.xls')
meta.head()

,NAME,WEB,CASH_BACK_HEIGHT,TRANCHE_STMT_COUNT,OFFER_TYPE,ADVERT_TEXT
0,Pro-bike,https://pro-bike.ru/,3,NaN,STANDARD,Pro-Bike - это специализированный велосипедный...


In [14]:
fill_preset(data, meta)

In [15]:
es = ElasticsearchRanker(preset=preset)

In [93]:
ranking = es.rank(
    {
        'Item': 'велосипед',
        'Attributes': '',
        'Price_from': 0,
        'Price_to': 1000000,
        'Cashback': 0
    }
)
ranking

[{'Item': 'велосипеды~шоссейный велосипед~no title',
  'Attributes': '',
  'Price': 167570,
  'Offer': 'Pro-bike',
  'Web': 'https://pro-bike.ru/',
  'Cashback': 3,
  'Period': 0,
  'Offer_type': 'STANDARD',
  'Advert_text': 'Pro-Bike - это специализированный велосипедный интернет магазин с более чем 10-летним опытом продаж. Компания базируется в Москве и осуществляет продажи по всей России. За прошедшее время продано более 20 000 велосипедов различных типовых. Среди клиентов вы можете встретить: профессиональных спортсменов, людей увлеченных велосипедов, тех, кто выбирает велосипед в качестве транспортного средства, молодых родителей, выбирающих велосипед для себя и своих детей.'},
 {'Item': 'велосипеды~шоссейный велосипед~no title',
  'Attributes': '',
  'Price': 167570,
  'Offer': 'Pro-bike',
  'Web': 'https://pro-bike.ru/',
  'Cashback': 3,
  'Period': 0,
  'Offer_type': 'STANDARD',
  'Advert_text': 'Pro-Bike - это специализированный велосипедный интернет магазин с более чем 10-лет

In [94]:
ranking = [r for r in ranking if 'Offer' in r]

In [95]:
unique_offers = {product['Offer'] for product in ranking}
offers = unique_offers
offers

{'Pro-bike', 'double sports'}

In [96]:
import itertools as it

s = [
    {
        'offer': offer,
        'products': list(products),  # here we can return shorten information about the 
                                    # products or only links to them
    }
    for offer, products in it.groupby(ranking, key=lambda product: product['Offer'])
]

In [99]:
[ss['offer'] for ss in s]

['Pro-bike', 'double sports']